# Introduction
In this project, I will be working on fraud detection project, a competition on kaggle by IEEE-Computational Intelligence Society. IEEE-CIS works across a variety of AI and machine learning areas, including deep neural networks, fuzzy systems, evolutionary computation, and swarm intelligence. Vesta corporation provided the dataset for this competition. Vesta corporation is the forerunner in guaranteed e-commerce payment solutions. In this dataset, we have more than 500000 observations in each dataset (train and test) with more than 400 columns. Many columns have missing values which is very common in the real world datasets. Dataset itself has many other issues like skewness, high correlation between independent variables. In this situation, we don't have to worry about missing values because I will use lightgbm/xgboost which will take care of this problem. The dataset is available [on this link](https://www.kaggle.com/c/ieee-fraud-detection/data).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, roc_auc_score
import pickle
warnings.filterwarnings(action = 'ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
backup_train_id = pd.read_csv('train_identity.csv')
backup_train_tran = pd.read_csv('train_transaction.csv')
backup_test_id = pd.read_csv('test_identity.csv')
backup_test_tran = pd.read_csv('test_transaction.csv')

In [3]:
train_id = backup_train_id.copy()
train_tran = backup_train_tran.copy()
test_id = backup_test_id.copy()
test_tran = backup_test_tran.copy()

## Merging dataset

In [4]:
train = pd.merge(train_tran, train_id, how = 'left', on = 'TransactionID')
test = pd.merge(test_tran, test_id, how = 'left', on = 'TransactionID')
test.columns = test.columns.str.replace("[-]", "_")
ntrain = train.shape[0]
y = train['isFraud']
train.drop(['isFraud'], axis = 1, inplace = True)
data = pd.concat([train, test], axis = 0)

In [5]:
data.shape

(1097231, 433)

In [6]:
missing_values = data.isnull().sum()/data.shape[0]
missing_values = missing_values[missing_values > 0.9]
missing_values = missing_values.sort_values(ascending = False)
missing_values = pd.DataFrame(data = missing_values)
missing_values = missing_values.rename(columns = {0:'Missing_Perc'})
missing_values

,Missing_Perc
id_24,0.991354
id_25,0.990730
id_26,0.990695
id_08,0.990691
id_07,0.990691
id_21,0.990687
id_27,0.990676
id_23,0.990676
id_22,0.990676
dist2,0.932500


## Removing columns with more than 90% missing values and columns with low variance
We can see from the above table that many columns have missing values more than 90%. These columns will not have that much impact on the outcome of the model.

In [7]:
high_missing_cols = list(missing_values.index)
low_var_cols =  [col for col in data.columns if data[col].value_counts(dropna=False,
                                                                         normalize=True).values[0] > 0.9]
remove_cols = list(set(high_missing_cols + low_var_cols))
len(remove_cols)

71

In [8]:
data_new = data.drop(remove_cols, axis = 1)
print("Data : ", data_new.shape)

Data :  (1097231, 362)


In [9]:
df = data_new.copy()

## Feature Engineering and Selection

In [10]:
df['TransactionAmt'] = np.log(df['TransactionAmt'])
df[['P_email1', 'P_email2', 'P_email3']] = df['P_emaildomain'].str.split('.', expand = True)
df[['R_email1', 'R_email2', 'R_email3']] = df['R_emaildomain'].str.split('.', expand = True)
df['Hour'] = (df['TransactionDT']//3600)%24
df['Weekday'] = ((df['TransactionDT']//3600)/24)%7
df.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V66,V67,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V89,V90,V91,V92,V93,V94,V95,V96,V97,V99,V100,V126,V127,V128,V130,V131,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V282,V283,V285,V287,V288,V289,V291,V292,V294,V296,V302,V303,V304,V306,V307,V308,V310,V312,V313,V314,V315,V317,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,P_email1,P_email2,P_email3,R_email1,R_email2,R_email3,Hour,Weekday
0,2987000,86400,4.226834,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0
1,2987001,86401,3.367296,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [11]:
df.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V66,V67,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V89,V90,V91,V92,V93,V94,V95,V96,V97,V99,V100,V126,V127,V128,V130,V131,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V282,V283,V285,V287,V288,V289,V291,V292,V294,V296,V302,V303,V304,V306,V307,V308,V310,V312,V313,V314,V315,V317,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,P_email1,P_email2,P_email3,R_email1,R_email2,R_email3,Hour,Weekday
0,2987000,86400,4.226834,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0
1,2987001,86401,3.367296,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

## Label Encoding

In [12]:
cat_features = df.dtypes[df.dtypes == 'object'].index
# numeric_features = df.dtypes[df.dtypes != 'object'].index
for i in cat_features:
#     df[i] = df[i].fillna('Missing')
    lb = LabelEncoder()
    df[i] = lb.fit_transform(df[i].astype('str'))
    

In [13]:
# numeric_data = df[numeric_features]
# numeric_data.shape

In [14]:
# correlation = numeric_data.corr().abs()

In [15]:
# plt.figure(figsize = (15,15))
# sns.heatmap(correlation, vmin = 0, vmax = 1, cmap='Reds')
# plt.title('Correlation', fontsize = 9)
# plt.show()

In [16]:
# high_corr = correlation.unstack().sort_values().drop_duplicates()
# high_corr = high_corr[(high_corr > 0.9) & (high_corr < 1.00)]
# high_corr = high_corr.sort_values(ascending = False)
# high_corr

In [17]:
df.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V66,V67,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V89,V90,V91,V92,V93,V94,V95,V96,V97,V99,V100,V126,V127,V128,V130,V131,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V282,V283,V285,V287,V288,V289,V291,V292,V294,V296,V302,V303,V304,V306,V307,V308,V310,V312,V313,V314,V315,V317,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,P_email1,P_email2,P_email3,R_email1,R_email2,R_email3,Hour,Weekday
0,2987000,86400,4.226834,4,13926,NaN,150.0,1,142.0,1,315.0,87.0,19.0,32,32,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,1,1,1,2,0,1,2,2,2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,3,2,NaN,NaN,NaN,2,2,86,136,NaN,461,4,2,2,2,2,2,2740,25,6,3,25,6,3,0,1.0
1,2987001,86401,3.367296,4,2755,404.0,150.0,2,102.0,1,325.0,87.0,NaN,16,32,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,2,2,2,0,1,1,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

## Remove High Correlated Features
In this dataset, we have many columns with correlation of more than 0.9. I will simply remove these columns.

In [18]:
columns_to_remove = ['TransactionID', 'P_emaildomain', 'DeviceInfo', 'R_emaildomain',
              'V300','V309','V111','V124','V106','V125','V315','V134','V102','V123','V316','V113',
              'V136','V305','V110','V299','V289','V286','V318','V304','V116','V284','V293',
              'V137','V295','V301','V104','V311','V115','V109','V119','V321','V114','V133','V122','V319',
              'V105','V112','V118','V117','V121','V108','V135','V320','V303','V297','V120',
              'V1','V14','V41','V65','V88', 'V89', 'V107', 'V68', 'V28', 'V27', 'V29', 'V241','V269',
              'V240', 'V325', 'V138', 'V154', 'V153', 'V330', 'V142', 'V195', 'V302', 'V328', 'V327', 
              'V198', 'V196', 'V155']
remove = list(set(columns_to_remove) - set(remove_cols))
X = df.drop(remove, axis=1)


In [19]:
X.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,C1,C2,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V66,V67,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V90,V91,V92,V93,V94,V95,V96,V97,V99,V100,V126,V127,V128,V130,V131,V139,V140,V141,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V197,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V282,V283,V285,V287,V288,V291,V292,V294,V296,V306,V307,V308,V310,V312,V313,V314,V317,V322,V323,V324,V326,V329,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,P_email1,P_email2,P_email3,R_email1,R_email2,R_email3,Hour,Weekday
0,86400,4.226834,4,13926,NaN,150.0,1,142.0,1,315.0,87.0,19.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,1,1,1,2,0,1,2,2,2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,3,2,NaN,NaN,NaN,2,2,86,136,NaN,461,4,2,2,2,2,2,25,6,3,25,6,3,0,1.0
1,86401,3.367296,4,2755,404.0,150.0,2,102.0,1,325.0,87.0,NaN,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,2,2,2,0,1,1,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

## Split the dataset

In [20]:
X_train = X[:ntrain]
X_test = X[ntrain:]
print(X_train.shape)
print(X_test.shape)
print(y.shape)

(590540, 342)
(506691, 342)
(590540,)


In [23]:
trainset = lgb.Dataset(data = X_train, label=y)

params =  {'learning_rate' : 0.001,
           'objective' : 'binary',
           'boosting_type' : 'gbdt',
           'subsample': 0.9,
           'num_leaves': 200,
           'max_depth' : 20, 
           'colsample_bytree': 0.9,
           'n_jobs': 3,
           'feature_fraction': 0.25,
           'bagging_freq': 1,
           'bagging_seed': 11,
           'bagging_fraction': 0.7,
           'boost_from_average': False,
           'max_bin':200
          
}

In [82]:


## lightgbm cv
cv_result = lgb.cv(params = params,
                   num_boost_round=15000,
                   nfold=5,
                   train_set=trainset,
                   early_stopping_rounds=100,
                   metrics='auc',
                   seed=42,
                   shuffle=True,
                   verbose_eval=True,
                   eval_train_metric=True)

[1]	cv_agg's train auc: 0.851533 + 0.00178869	cv_agg's valid auc: 0.848588 + 0.00318975
[2]	cv_agg's train auc: 0.868219 + 0.000936871	cv_agg's valid auc: 0.864769 + 0.0019475
[3]	cv_agg's train auc: 0.870478 + 0.000649536	cv_agg's valid auc: 0.866985 + 0.00229796
[4]	cv_agg's train auc: 0.873932 + 0.000381798	cv_agg's valid auc: 0.870456 + 0.00267789
[5]	cv_agg's train auc: 0.875983 + 0.000521773	cv_agg's valid auc: 0.872436 + 0.00260727
[6]	cv_agg's train auc: 0.877148 + 0.000510838	cv_agg's valid auc: 0.873722 + 0.00213866
[7]	cv_agg's train auc: 0.878633 + 0.000550852	cv_agg's valid auc: 0.875221 + 0.00228442
[8]	cv_agg's train auc: 0.879157 + 0.000495395	cv_agg's valid auc: 0.875572 + 0.00224204
[9]	cv_agg's train auc: 0.87992 + 0.000628974	cv_agg's valid auc: 0.876217 + 0.00239136
[10]	cv_agg's train auc: 0.880838 + 0.000508227	cv_agg's valid auc: 0.877072 + 0.00229089
[11]	cv_agg's train auc: 0.881579 + 0.000464587	cv_agg's valid auc: 0.877882 + 0.00244614
[12]	cv_agg's train au

[93]	cv_agg's train auc: 0.88682 + 0.000333242	cv_agg's valid auc: 0.882578 + 0.00184594
[94]	cv_agg's train auc: 0.886729 + 0.000341137	cv_agg's valid auc: 0.882496 + 0.00184679
[95]	cv_agg's train auc: 0.886711 + 0.000347247	cv_agg's valid auc: 0.882491 + 0.00182701
[96]	cv_agg's train auc: 0.886696 + 0.000337074	cv_agg's valid auc: 0.882455 + 0.0018388
[97]	cv_agg's train auc: 0.886724 + 0.000338952	cv_agg's valid auc: 0.882481 + 0.00183838
[98]	cv_agg's train auc: 0.886803 + 0.000338774	cv_agg's valid auc: 0.882562 + 0.00185384
[99]	cv_agg's train auc: 0.886815 + 0.000343727	cv_agg's valid auc: 0.882586 + 0.00186774
[100]	cv_agg's train auc: 0.886857 + 0.000332839	cv_agg's valid auc: 0.882653 + 0.00189079
[101]	cv_agg's train auc: 0.886799 + 0.000343642	cv_agg's valid auc: 0.882597 + 0.00188853
[102]	cv_agg's train auc: 0.886787 + 0.000349207	cv_agg's valid auc: 0.882593 + 0.00188312
[103]	cv_agg's train auc: 0.886826 + 0.000352581	cv_agg's valid auc: 0.882641 + 0.00191217
[104]	cv

[184]	cv_agg's train auc: 0.887694 + 0.000383574	cv_agg's valid auc: 0.883533 + 0.00194827
[185]	cv_agg's train auc: 0.887736 + 0.00037004	cv_agg's valid auc: 0.883561 + 0.00195247
[186]	cv_agg's train auc: 0.887685 + 0.000363007	cv_agg's valid auc: 0.883519 + 0.0019478
[187]	cv_agg's train auc: 0.887692 + 0.000362775	cv_agg's valid auc: 0.883536 + 0.001952
[188]	cv_agg's train auc: 0.887665 + 0.000355786	cv_agg's valid auc: 0.883512 + 0.00193964
[189]	cv_agg's train auc: 0.887664 + 0.000356702	cv_agg's valid auc: 0.883522 + 0.00193845
[190]	cv_agg's train auc: 0.88766 + 0.000355879	cv_agg's valid auc: 0.883521 + 0.00193866
[191]	cv_agg's train auc: 0.887638 + 0.000350969	cv_agg's valid auc: 0.883489 + 0.00192747
[192]	cv_agg's train auc: 0.887652 + 0.000348586	cv_agg's valid auc: 0.883491 + 0.00192429
[193]	cv_agg's train auc: 0.887665 + 0.000346959	cv_agg's valid auc: 0.883494 + 0.00193063
[194]	cv_agg's train auc: 0.8877 + 0.000335699	cv_agg's valid auc: 0.883526 + 0.00194124
[195]	

[275]	cv_agg's train auc: 0.888524 + 0.000357473	cv_agg's valid auc: 0.884144 + 0.00201155
[276]	cv_agg's train auc: 0.888522 + 0.000361648	cv_agg's valid auc: 0.884144 + 0.0020143
[277]	cv_agg's train auc: 0.888522 + 0.000356192	cv_agg's valid auc: 0.884145 + 0.00202186
[278]	cv_agg's train auc: 0.888555 + 0.000355486	cv_agg's valid auc: 0.884176 + 0.00202735
[279]	cv_agg's train auc: 0.888589 + 0.000354918	cv_agg's valid auc: 0.884214 + 0.00203686
[280]	cv_agg's train auc: 0.888583 + 0.000359029	cv_agg's valid auc: 0.884208 + 0.00204404
[281]	cv_agg's train auc: 0.888602 + 0.000362881	cv_agg's valid auc: 0.884224 + 0.00202338
[282]	cv_agg's train auc: 0.888574 + 0.000363719	cv_agg's valid auc: 0.884201 + 0.0020201
[283]	cv_agg's train auc: 0.888587 + 0.000367625	cv_agg's valid auc: 0.884204 + 0.002013
[284]	cv_agg's train auc: 0.888595 + 0.00036986	cv_agg's valid auc: 0.88421 + 0.00201319
[285]	cv_agg's train auc: 0.888606 + 0.000367434	cv_agg's valid auc: 0.884214 + 0.00201976
[286]

[366]	cv_agg's train auc: 0.889636 + 0.000341963	cv_agg's valid auc: 0.885084 + 0.00199199
[367]	cv_agg's train auc: 0.889663 + 0.000339823	cv_agg's valid auc: 0.885107 + 0.0019932
[368]	cv_agg's train auc: 0.889663 + 0.000340653	cv_agg's valid auc: 0.885102 + 0.00198897
[369]	cv_agg's train auc: 0.889683 + 0.000333983	cv_agg's valid auc: 0.885117 + 0.00199782
[370]	cv_agg's train auc: 0.889698 + 0.000336295	cv_agg's valid auc: 0.885135 + 0.00200703
[371]	cv_agg's train auc: 0.889696 + 0.000336081	cv_agg's valid auc: 0.885135 + 0.00201072
[372]	cv_agg's train auc: 0.889722 + 0.000342027	cv_agg's valid auc: 0.885167 + 0.00201041
[373]	cv_agg's train auc: 0.889703 + 0.000341008	cv_agg's valid auc: 0.885152 + 0.00200943
[374]	cv_agg's train auc: 0.889704 + 0.000340835	cv_agg's valid auc: 0.885153 + 0.00201027
[375]	cv_agg's train auc: 0.889711 + 0.000335434	cv_agg's valid auc: 0.885156 + 0.00200932
[376]	cv_agg's train auc: 0.889726 + 0.000337639	cv_agg's valid auc: 0.885172 + 0.00200334


...

[14556]	cv_agg's train auc: 0.99469 + 0.000103838	cv_agg's valid auc: 0.964692 + 0.00103866
[14557]	cv_agg's train auc: 0.994691 + 0.000103824	cv_agg's valid auc: 0.964695 + 0.00103809
[14558]	cv_agg's train auc: 0.994692 + 0.000103495	cv_agg's valid auc: 0.964696 + 0.00103797
[14559]	cv_agg's train auc: 0.994692 + 0.000103437	cv_agg's valid auc: 0.964698 + 0.00103751
[14560]	cv_agg's train auc: 0.994693 + 0.000103429	cv_agg's valid auc: 0.964698 + 0.00103773
[14561]	cv_agg's train auc: 0.994694 + 0.000103551	cv_agg's valid auc: 0.9647 + 0.00103739
[14562]	cv_agg's train auc: 0.994695 + 0.000103504	cv_agg's valid auc: 0.964701 + 0.00103754
[14563]	cv_agg's train auc: 0.994696 + 0.000103415	cv_agg's valid auc: 0.964702 + 0.00103756
[14564]	cv_agg's train auc: 0.994697 + 0.000103288	cv_agg's valid auc: 0.964703 + 0.00103816
[14565]	cv_agg's train auc: 0.994699 + 0.000103251	cv_agg's valid auc: 0.964704 + 0.00103815
[14566]	cv_agg's train auc: 0.9947 + 0.000103209	cv_agg's valid auc: 0.96

[14645]	cv_agg's train auc: 0.994779 + 0.000104175	cv_agg's valid auc: 0.964784 + 0.00103641
[14646]	cv_agg's train auc: 0.994781 + 0.000104179	cv_agg's valid auc: 0.964785 + 0.00103656
[14647]	cv_agg's train auc: 0.994781 + 0.000104191	cv_agg's valid auc: 0.964786 + 0.00103649
[14648]	cv_agg's train auc: 0.994782 + 0.000104185	cv_agg's valid auc: 0.964788 + 0.0010365
[14649]	cv_agg's train auc: 0.994783 + 0.000104109	cv_agg's valid auc: 0.964789 + 0.00103646
[14650]	cv_agg's train auc: 0.994783 + 0.000104186	cv_agg's valid auc: 0.96479 + 0.0010368
[14651]	cv_agg's train auc: 0.994784 + 0.000103873	cv_agg's valid auc: 0.964791 + 0.00103664
[14652]	cv_agg's train auc: 0.994785 + 0.000103916	cv_agg's valid auc: 0.964792 + 0.00103664
[14653]	cv_agg's train auc: 0.994786 + 0.000103845	cv_agg's valid auc: 0.964793 + 0.00103656
[14654]	cv_agg's train auc: 0.994787 + 0.000103567	cv_agg's valid auc: 0.964794 + 0.00103694
[14655]	cv_agg's train auc: 0.994788 + 0.0001033	cv_agg's valid auc: 0.96

[14734]	cv_agg's train auc: 0.994867 + 0.000102383	cv_agg's valid auc: 0.964881 + 0.00103815
[14735]	cv_agg's train auc: 0.994868 + 0.000102419	cv_agg's valid auc: 0.964882 + 0.00103797
[14736]	cv_agg's train auc: 0.994869 + 0.000102835	cv_agg's valid auc: 0.964882 + 0.00103863
[14737]	cv_agg's train auc: 0.99487 + 0.000102768	cv_agg's valid auc: 0.964883 + 0.00103872
[14738]	cv_agg's train auc: 0.994871 + 0.000102921	cv_agg's valid auc: 0.964884 + 0.00103893
[14739]	cv_agg's train auc: 0.994871 + 0.00010264	cv_agg's valid auc: 0.964885 + 0.0010391
[14740]	cv_agg's train auc: 0.994872 + 0.000102557	cv_agg's valid auc: 0.964886 + 0.00103889
[14741]	cv_agg's train auc: 0.994873 + 0.000102582	cv_agg's valid auc: 0.964887 + 0.00103895
[14742]	cv_agg's train auc: 0.994873 + 0.000102785	cv_agg's valid auc: 0.964887 + 0.00103903
[14743]	cv_agg's train auc: 0.994874 + 0.000102709	cv_agg's valid auc: 0.964889 + 0.00103915
[14744]	cv_agg's train auc: 0.994875 + 0.000102712	cv_agg's valid auc: 0.

[14823]	cv_agg's train auc: 0.994943 + 9.92345e-05	cv_agg's valid auc: 0.964967 + 0.00103327
[14824]	cv_agg's train auc: 0.994944 + 9.9271e-05	cv_agg's valid auc: 0.964968 + 0.0010331
[14825]	cv_agg's train auc: 0.994944 + 9.92046e-05	cv_agg's valid auc: 0.964969 + 0.00103335
[14826]	cv_agg's train auc: 0.994946 + 9.83982e-05	cv_agg's valid auc: 0.96497 + 0.00103411
[14827]	cv_agg's train auc: 0.994947 + 9.84587e-05	cv_agg's valid auc: 0.964971 + 0.00103376
[14828]	cv_agg's train auc: 0.994948 + 9.85059e-05	cv_agg's valid auc: 0.964972 + 0.00103357
[14829]	cv_agg's train auc: 0.994948 + 9.85502e-05	cv_agg's valid auc: 0.964973 + 0.00103348
[14830]	cv_agg's train auc: 0.994949 + 9.83514e-05	cv_agg's valid auc: 0.964974 + 0.00103307
[14831]	cv_agg's train auc: 0.99495 + 9.80475e-05	cv_agg's valid auc: 0.964974 + 0.0010331
[14832]	cv_agg's train auc: 0.994951 + 9.77633e-05	cv_agg's valid auc: 0.964975 + 0.00103311
[14833]	cv_agg's train auc: 0.994953 + 9.818e-05	cv_agg's valid auc: 0.9649

[14912]	cv_agg's train auc: 0.995022 + 9.86002e-05	cv_agg's valid auc: 0.965051 + 0.00102901
[14913]	cv_agg's train auc: 0.995023 + 9.8607e-05	cv_agg's valid auc: 0.965052 + 0.00102863
[14914]	cv_agg's train auc: 0.995023 + 9.85497e-05	cv_agg's valid auc: 0.965053 + 0.00102836
[14915]	cv_agg's train auc: 0.995024 + 9.86244e-05	cv_agg's valid auc: 0.965053 + 0.00102843
[14916]	cv_agg's train auc: 0.995026 + 9.8822e-05	cv_agg's valid auc: 0.965054 + 0.00102892
[14917]	cv_agg's train auc: 0.995027 + 9.87696e-05	cv_agg's valid auc: 0.965055 + 0.00102929
[14918]	cv_agg's train auc: 0.995028 + 9.89628e-05	cv_agg's valid auc: 0.965056 + 0.00102916
[14919]	cv_agg's train auc: 0.995028 + 9.89873e-05	cv_agg's valid auc: 0.965057 + 0.00102915
[14920]	cv_agg's train auc: 0.995029 + 9.91415e-05	cv_agg's valid auc: 0.965058 + 0.0010291
[14921]	cv_agg's train auc: 0.99503 + 9.90234e-05	cv_agg's valid auc: 0.965058 + 0.00102873
[14922]	cv_agg's train auc: 0.995031 + 9.90047e-05	cv_agg's valid auc: 0.9

Model didnt use early stopping which means we can use more number of boosting rounds and our model can be improved. But in this case I will use 15000 iterations in this case as the dataset is big and it takes a lot of time to train the model. It took almost 2 hours to complete 15000 iterations. 

In [24]:
best_iteration = 15000

model = lgb.train(params = params, 
                  train_set = trainset,
                  num_boost_round = best_iteration
)

In [26]:
pred = model.predict(X_test)

In [27]:
## prepare dataset for submission
submission = pd.DataFrame()
submission['TransactionID'] = test['TransactionID']
submission['isFraud'] = pred

In [28]:
## save predictions for submission
submission.to_csv('submission2.csv', index = False)

In [29]:
## Save model
pickle.dump(model, open('lightgbm.sav', 'wb'))

Final result on the leaderboard is 0.916. Top score is 0.945. With better preprocessing, feature selection and combining lightgbm with xgboost/catboost, we can achieve higher auc. Thank you for checking out my kernel. Any suggestion and criticism is welcomed.